# a quick-start notebook for ChiantiPy

## similar to the quick-start guide in the online documentation

In [1]:
import os

In [2]:
os.environ['XUVTOP']

'/data2/a2git/chdbase/'

This short tutorial will demonstrate some of the capabilities of ChiantiPy and the CHIANTI database. 
It assumes that you know what the CHIANTI database provides and why you want to use it. 
The current implementation uses Version 0.6.4 of ChiantiPy and version 8.0 of the CHIANTI database and mainly provides access to methods concerned with single ions. An ion such as Fe XIV is specified by the string ‘fe_14’, in the usual CHIANTI notation.

Bring up a Jupyter notebook

    > jupyter notebook

It is easiest to do this in the directory where the notebook (.ipynb) resides.

Once you open QuickStart.ipynb in the browser window, you will see something like this.

In my Ipython 00-setup.py file I have already performed

In [3]:
import ChiantiPy
import ChiantiPy.core as ch
import ChiantiPy.tools.filters as chfilters
import ChiantiPy.tools.io as chio
import ChiantiPy.tools.data as chdata
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [4]:
matplotlib qt

In [5]:
autoreload 3

In [6]:
ChiantiPy.__version__

'0.15.2'

In [7]:
import IPython
print(' IPython version = %i.%i.%i'%(IPython.version_info[0],IPython.version_info[1],IPython.version_info[2]))

 IPython version = 8.37.0


It is useful to open a qtconsole where are the calculations can be easily examined

## if you have ipyparallel installed it is necessary to do the following before opening a qtconsole

In [8]:
import ipyparallel
ipyparallel.bind_kernel = lambda : None

In [9]:
qtconsole

In [10]:
chianti_version = chio.versionRead()

In [11]:
chianti_version

'11.0.2'

# setting default values

### ChiantiPy determines a number of default setting on instantiation. Without doing anything, the default values

### listed below will be used,

| **setting**     |**default**                     | **possible values**      |
---------- | --------------------------- | ---------------------|
|wavelength | angstrom                    | angstrom, nm, ev, kev|
|flux       | energy                      | energy, photon       |
|abundfile  | sun_photospheric_2015_scott | any.abund            |
|ioneqfile  | chianti                     | any.ioneq            |

### to use any of the other possible values, check out the notes/setting_default_values in the documentation

the defaults can be checked

In [12]:
chdata.Defaults.keys()

dict_keys(['abundfile', 'ioneqfile', 'wavelength', 'flux', 'gui', 'rcfile'])

In [13]:
chdata.Defaults['wavelength']

'angstrom'

## or, defaults can be set in a chiantirc file in ~/.config - a  basic one is included in the distribution

## the default mpl rcParams for several properties

In [52]:
propt = ['xtick.labelsize', 'ytick.labelsize', 'axes.labelsize', 'axes.linewidth', 'grid.linewidth', 'legend.fontsize', 'axes.titlesize', 'font.size']

for aprop in propt:
    print('%s  %s'%(aprop, mpl.rcParams[aprop]))

xtick.labelsize  16.0
ytick.labelsize  16.0
axes.labelsize  16.0
axes.linewidth  1.5
grid.linewidth  1.5
legend.fontsize  large
axes.titlesize  16.0
font.size  12.0


## setting some matplot parameters for pretty plotting

In [30]:
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['axes.labelsize'] = 16
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams['grid.linewidth'] = 1.5
mpl.rcParams['legend.fontsize'] = 'large'
mpl.rcParams['axes.titlesize'] = 16
mpl.rcParams['font.size'] = 12

Level populations
=================

As a start, we will examine the various properties of the Fe XIV emissivities as a function of temperature and density. So, let’s define a numpy array of temperatures and a value for the electron density

In [19]:
temp = 10.**(5.8 + 0.05*np.arange(21.))
edens = 1.e+9

In ChiantiPy, temperatures are currently given in degrees Kelvin and densities as the number electron density per cubic cm. Then, construct fe14 as would be typically done

In [20]:
fe14 = ch.ion('fe_14', temperature=temp, eDensity=edens, em=1.e+27)

alternately, since **temperature** and **eDensity** are the first keyword arguments
fe14 = ch.ion('fe_14', temp, edens)

In [21]:
fe14.popPlot(top = 10, addLegend=False)

produces a matplotlib plot window were the population of the top 10 (the default) levels are plotted as a function of temperature.

If the level populations had not already been calculated, popPlot() would have invoked the populate() method which calculates the level populations and stores them in the Population dictionary, with keys = [‘protonDensity’, ‘population’, ‘temperature’, ‘density’].

The populations vs. temperature is not particularly interesting.  Plotting them vs. density is more interesting and will come latere

A ChiantiPy Convention
======================

Classes and function of ChiantiPy start with lower case letters. 
----------------------------------------------------------------

Data attached to the instantiation of a class will start with a capital letter. For example,
--------------------------------------------------------------------------------------------

fe14.populate() creates fe14.Population containing the level population information

fe14.emiss() creates fe14.Emiss containing the line emissivity information

fe14.intensity() creates fe14.Intensity containing the line intensity information that includes the elemental abundance and the ionization equilibrium

fe14.spectrum() creates fe14.Spectrum contain the line and continuum spectrum information

## it is possible to get some basic information about an ion

In [22]:
nameDict = chutil.convertName('fe_12')

In [23]:
nameDict

{'Z': 26,
 'Ion': 12,
 'Zion': 11,
 'electrons': 15,
 'Dielectronic': False,
 'Element': 'Fe',
 'higher': 'fe_13',
 'lower': 'fe_11',
 'filename': '/data2/a2git/chdbase/fe/fe_12/fe_12',
 'iso': 15,
 'isoEl': 'P',
 'spectroscopic': 'Fe XII',
 'experimental': 'Fe 11+'}

# Attributes of an ion model

In [24]:
fe12 = ch.ion('fe_12', setup = False)

a list of attributes can be found using dir(), but this is kind of messy.  A list is provided below

* AbundAll
* Abundance
* Defaults
* Dielectronic
* FIP
* FileName
* GrndLevels
* HigherName
* Ion
* IonStr
* Ip
* Iso
* Labels
* RStar
* RadTemperature
* Spectroscopic
* Z

In [25]:
print('nuclear charge:  %i'%(fe12.Z))

nuclear charge:  26


In [26]:
print('ionization potential   %6.2f  eV'%(fe12.Ip))

ionization potential   330.79  eV


In [27]:
print('isoelectronic sequence %i, the number of electrons'%(fe12.Iso))

isoelectronic sequence 15, the number of electrons


Spectral Line Intensities
=========================

In [28]:
fe14.intensity()
for akey in sorted(fe14.Intensity):
    print('%10s'%(akey))

    avalue
        em
integrated
 intensity
      ionS
      lvl1
      lvl2
       obs
   pretty1
   pretty2
       wvl
    xlabel
    ylabel


the units for line intensities, the key **intensity**, are erg cm$^{-2}$ s$^{-1}$ sr$^{-1}$ / $\int N_e N_H d\ell $, if the emission meassure **em** *not* unspecified

In [31]:
fe14.intensityPlot(wvlRange=[210.,220.])

using index =    10 specifying temperature =    2.00e+06


In [32]:
index = 11
print('temperature = %10.2e'%(temp[index]))

temperature =   2.24e+06


In [33]:
fe14.intensityPlot(index=11, wvlRange=[210.,220.])

using index =    11 specifying temperature =    2.24e+06


In [34]:
fe14.intensityPlot(index=10, wvlRange=[210.,220.], relative=True, doTitle=False, lw=2)

using index =    10 specifying temperature =    2.00e+06


In [35]:
fe14.intensityList(wvlRange=[210.,220.])

using index =    10 specifying temperature =    2.00e+06
   
 ------------------------------------------
   
  Ion  lvl1  lvl2                     lower -                     upper      Wvl (Å)    Intensity      A value Obs
fe_14     1    11              3s2.3p 2P0.5 - 3s2.3d 2D1.5                  211.3170     2.24e+02     3.81e+10 Y
fe_14     4    27              3s.3p2 4P1.5 - 3s.3p(3P).3d 4P1.5            212.1250     5.12e-01     2.21e+10 Y
fe_14     4    28              3s.3p2 4P1.5 - 3s.3p(3P).3d 4D2.5            212.1680     3.86e-01     1.15e+10 Y
fe_14     3    24              3s.3p2 4P0.5 - 3s.3p(3P).3d 4D0.5            213.1950     7.73e-01     4.26e+10 Y
fe_14     3    23              3s.3p2 4P0.5 - 3s.3p(3P).3d 4D1.5            213.8820     1.33e+00     2.97e+10 Y
fe_14     5    28              3s.3p2 4P2.5 - 3s.3p(3P).3d 4D2.5            216.5780     9.32e-01     2.83e+10 Y
fe_14     5    25              3s.3p2 4P2.5 - 3s.3p(3P).3d 4D3.5            216.9170     1.66e+00 

here, we have used the default value for the keyword argument **top** which specified how many of the most intense lines to list.  Also, **index** can be specified to give a different temperature, and **relative** can be set to 1 give relative emissivities

In [36]:
fe14.intensityList(wvlRange=[210.,220.], relative=1, index=11)

using index =    11 specifying temperature =    2.24e+06
   
 ------------------------------------------
   
  Ion  lvl1  lvl2                     lower -                     upper      Wvl (Å)    Intensity      A value Obs
fe_14     1    11              3s2.3p 2P0.5 - 3s2.3d 2D1.5                  211.3170     1.00e+00     3.81e+10 Y
fe_14     4    27              3s.3p2 4P1.5 - 3s.3p(3P).3d 4P1.5            212.1250     2.27e-03     2.21e+10 Y
fe_14     4    28              3s.3p2 4P1.5 - 3s.3p(3P).3d 4D2.5            212.1680     1.69e-03     1.15e+10 Y
fe_14     3    24              3s.3p2 4P0.5 - 3s.3p(3P).3d 4D0.5            213.1950     3.39e-03     4.26e+10 Y
fe_14     3    23              3s.3p2 4P0.5 - 3s.3p(3P).3d 4D1.5            213.8820     5.89e-03     2.97e+10 Y
fe_14     5    28              3s.3p2 4P2.5 - 3s.3p(3P).3d 4D2.5            216.5780     4.08e-03     2.83e+10 Y
fe_14     5    25              3s.3p2 4P2.5 - 3s.3p(3P).3d 4D3.5            216.9170     7.09e-03 

optionally, an output file could also be created by setting the keyword **outFile** to the name of the desired name

The effect of electron density in line intensities
----------------------------------------------------

In [41]:
mpl.rcParams['legend.fontsize'] = 'small'

In [37]:
temp = 2.e+6
dens = 10.**(6. + 0.1*np.arange(61))
fe14 = ch.ion('fe_14', temp, dens)
fe14.popPlot()

In [38]:
mpl.rcParams['legend.fontsize'] = 'large'

a plot of the population of the top 10 levels is produced as a function of the electron density

In [39]:
fe14.intensityRatio(wvlRange=[210.,220.])

 ndens =    61 ntemp =     1
   219.130 3s2.3p 2P1.5 - 3s2.3d 2D2.5
   218.572 3s.3p2 4P1.5 - 3s.3p(3P).3d 4P2.5
   218.176 3s.3p2 2D2.5 - 3s.3p(3P).3d 2F3.5
   216.917 3s.3p2 4P2.5 - 3s.3p(3P).3d 4D3.5
   216.578 3s.3p2 4P2.5 - 3s.3p(3P).3d 4D2.5
   213.882 3s.3p2 4P0.5 - 3s.3p(3P).3d 4D1.5
   213.195 3s.3p2 4P0.5 - 3s.3p(3P).3d 4D0.5
   212.168 3s.3p2 4P1.5 - 3s.3p(3P).3d 4D2.5
   212.125 3s.3p2 4P1.5 - 3s.3p(3P).3d 4P1.5
   211.317 3s2.3p 2P0.5 - 3s2.3d 2D1.5


to obtain ratios of lines widely separated in wavelength, the **wvlRanges** keyword can be used

In [40]:
temp = 10.**(5.8 + 0.05*np.arange(21.))
edens = 1.e+9
fe12 = ch.ion('fe_12', temperature=temp, eDensity=edens)
fe12.intensityRatio(wvlRanges=[[190.,200.],[1240.,1250.]])

 ndens =     1 ntemp =    21
  1242.005 3s2 3p3 4S1.5 - 3s2 3p3 2P1.5
   196.921 3s2 3p3 2D2.5 - 3s2 3p2 3d 2D1.5
   196.640 3s2 3p3 2D2.5 - 3s2 3p2 3d 2D2.5
   195.179 3s2 3p3 2D1.5 - 3s2 3p2 3d 2D1.5
   195.119 3s2 3p3 4S1.5 - 3s2 3p2 3d 4P2.5
   193.509 3s2 3p3 4S1.5 - 3s2 3p2 3d 4P1.5
   192.394 3s2 3p3 4S1.5 - 3s2 3p2 3d 4P0.5
   191.910 3s 3p4 4P0.5 - 3s 3p3 3d 4D1.5
   191.049 3s2 3p3 2P1.5 - 3s2 3p2 3d 2D2.5
   190.068 3s2 3p3 4S1.5 - 3s2 3p2 3d 2D1.5


the most recently calculated intensity ratio is stored in fe14.IntensityRatio, with the following keys:

In [41]:
for akey in sorted(fe14.IntensityRatio):
    print('%10s'%(akey))

    denIdx
      desc
  eDensity
  filename
    numIdx
     ratio
temperature


G(n,T) function or G(T)
=======================

When G(n,T), for specific spectral line, is multiplied by the line of sight **emission measure**, $\int$ n$_e$ n_$H$ d $\ell$, it provides the predicted value of the line intensity in units of erg cm$^{-2}$ s$^{-1}$ sr$^{-1}$, if the value for **flux** in the** Defaults** is set to **energy** (the default value).  If **flux** is set to photon, the intensity is given in units of


photons cm$^{-2}$ s$^{-1}$ sr$^{-1}$.

A **chiantirc** file is included with the ChiantiPy distribution. If it is placed in $HOME/.chianti it will be read when ChiantiPy is initiated.  Editing this file allows you to specify the values of **flux** that you want and other things such as the set of elemental abundances and the ionization equilibrium.

In [42]:
temp = 10.**(5.8 + 0.05*np.arange(21.))
dens = 1.e+9
fe14 = ch.ion('fe_14', temp, dens)

In [43]:
fe14.gofnt(wvlRange=[210., 220.],top=3)

 ndens =     1 ntemp =    21


this brings up a plot of relative line ratios vs temperature and a single selection widget, similar to the intensityRatio process.  Multiple lines can be selected with the *control* key and their G(T) functions will summed.

The **g(n,T)** calculation is stored in the **Gofnt** dictionary, with keys

In [44]:
sorted(fe14.Gofnt.keys())

['eDensity',
 'gofnt',
 'index',
 'temperature',
 'transition',
 'wvl',
 'xlabel',
 'ylabel']

In [45]:
fe14.Gofnt['wvl']

array([211.317])

while this is a fairly straightforward way to get a G(T) function, it is not very practical to use for a more than a handful of lines.  For if the fe_14 line at 211.3172 is in a list of lines to be analyzed, a more practical way is the following

In [46]:
fe14.intensity()
dist = np.abs(np.asarray(fe14.Intensity['wvl']) - 211.3172)
idx = np.argmin(dist)
print(' wvl = %10.3f '%(fe14.Intensity['wvl'][idx]))

 wvl =    211.317 


In [47]:
plt.loglog(temp,fe14.Intensity['intensity'][:,idx], 'k')

once the axes are properly scaled, this produces the same values as fe14.Gofnt['gofnt']

# Ionization Equilibrium

For the Fe XIV example, the temperature was chosen to center around 2.e+6. It was not immediately apparent why this was done but in most of the following examples it is necessary to pick an appropriate temperature. This can be done with the ioneq class. To look at the ionization equilibrium for the iron ions (Z = 26, or ‘fe’)

In [56]:
mpl.rcParams['font.size'] = 14

In [57]:
fe = ch.ioneq(26)
fe.load()
fe.plot()
plt.tight_layout()

brings up a plot showing the ionization equilibrium for all of the stages of iron as a function of temperature

### there are now another way to plot the ionization equililibrium:  .plotObj

### both .plot and .plotObj return the fig and ax instances in the attribute IoneqPlotOjb (a dict)

This is pretty crowded and we are only interested in Fe XIV (fe_14), so

In [60]:
fe.plotObj(stages=[13,14,15],tRange=[1.e+6, 6.e+6], yr = [1.e-2, 0.4])

produces a plot of the ionization equilibria of Fe XIII, XIV and XV over a limited temperature range (tRange) and vertical range (yr)

In [59]:
fe.IoneqPlotObj.keys()

dict_keys(['fig', 'ax'])

In [51]:
ax = fe.IoneqPlotObj['ax']
ax.set_title('my title', fontsize=20)

Text(0.5, 1.0, 'my title')

In [61]:
mpl.rcParams['font.size'] = 12

## Calculating a New Ionization Equilibrium

The ionization equilibrium file in the CHIANTI distribution (chianti.ioneq) is calculated in the low-density limit for 121 temperatures
between 10$^4$ and 10$^9$ K.  For example, if you are interested in temperatures below 10$^4$ K, it is possible to use the ioneqMake
functions.

In [62]:
temp = np.logspace(3.5, 5.0, 16)
filename = 'my.ioneq'
directory = os.path.join(os.environ['XUVTOP'], 'ioneq')
reference = ['myself', '2025']
ch.Ioneq.ioneqMake(temperature = temp, filename = filename, directory = directory, reference = reference)

places a new ionization equilibrium file ( my.ioneq ) is placed in the standard CHIANTI directory for ionization equilibrium.
The only required keyword argument is the filename.  If the directory is not specified, the new ioneq file is placed in your home directory.
This calculates the ionization equilibrium for all ions from H through Zn.

the new ioneq file can be plotted as just above:

In [63]:
fig, ax = plt.subplots(layout = 'tight')
carbon = ch.ioneq(6)
carbon.load('my.ioneq')
carbon.plot(tRange = [3.e+3, 1.e+5])

Intensity Ratios
================

In [64]:
temp = 10.**(5.8 + 0.05*np.arange(21.))
dens = 1.e+9

In [65]:
fe14 = ch.ion('fe_14', temperature = temp, eDensity = dens)

In [66]:
fe14.intensityRatio(wvlRange=[210., 225.])

 ndens =     1 ntemp =    21
   224.355 3s.3p2 2D1.5 - 3s.3p(3P).3d 2F2.5
   223.232 3s.3p2 2P1.5 - 3s.3p(1P).3d 2D2.5
   221.109 3s.3p2 2P0.5 - 3s.3p(1P).3d 2D1.5
   220.084 3s2.3p 2P1.5 - 3s2.3d 2D1.5
   219.130 3s2.3p 2P1.5 - 3s2.3d 2D2.5
   218.572 3s.3p2 4P1.5 - 3s.3p(3P).3d 4P2.5
   218.176 3s.3p2 2D2.5 - 3s.3p(3P).3d 2F3.5
   216.917 3s.3p2 4P2.5 - 3s.3p(3P).3d 4D3.5
   213.882 3s.3p2 4P0.5 - 3s.3p(3P).3d 4D1.5
   211.317 3s2.3p 2P0.5 - 3s2.3d 2D1.5


this brings up a plot showing the relative emissivities on the top 10 Fe XIV lines
followed by a dialog where you can selector the numerator(s) and denominator(s) of the desired intensity ratio
so the specified ratio is then plotted
the intensityRatio as a function of temperature and density can be saved to a text file with the following

In [64]:
fe14.intensityRatioSave()

 saving ratio to filename = 224.355-___211.317.rat


In [65]:
fe14.intensityRatioSave(outFile='myratio.txt')

saves the ratio to the ascii file 'myratio.txt'

or, the intensity ratio as a function of electron density

In [67]:
temp = 2.e+6
dens = 10.**(6. + 0.1*np.arange(61))
fe14 = ch.ion('fe_14', temp, dens)

In [68]:
fe14.intensityRatio(wvlRange=[210.,220.], top = 5)

 ndens =    61 ntemp =     1
   219.130 3s2.3p 2P1.5 - 3s2.3d 2D2.5
   218.572 3s.3p2 4P1.5 - 3s.3p(3P).3d 4P2.5
   218.176 3s.3p2 2D2.5 - 3s.3p(3P).3d 2F3.5
   216.917 3s.3p2 4P2.5 - 3s.3p(3P).3d 4D3.5
   211.317 3s2.3p 2P0.5 - 3s2.3d 2D1.5


Spectrum of a single ion
========================

In [69]:
temp = 10.**(5.8 + 0.05*np.arange(21.))
edens = 1.e+9

In [70]:
fe14 = ch.ion('fe_14', temperature = 2.e+6, eDensity = edens, em=1.e+27)

In [71]:
wvl = 200. + 0.125*np.arange(801)
fe14.spectrum(wvl)

In [72]:
plt.figure()
plt.plot(wvl, fe14.Spectrum['intensity'])
xy = plt.axis()
xy

(np.float64(195.0),
 np.float64(305.0),
 np.float64(-20.277696703773966),
 np.float64(425.83163077925326))

## or with the newer object scheme

In [73]:
fig, ax = plt.subplots(tight_layout = True)
ax.plot(wvl, fe14.Spectrum['intensity'], lw = 2)
ax.set_ylim(bottom = 0.)

(0.0, 425.83163077925326)

In [74]:
ax.set_xlim(left = 200., right = 300.)
ax.set_ylim(bottom = 0., top = 500.)
ax.set_xlabel(fe14.Spectrum['xlabel'], fontsize=14)
ax.set_ylabel(fe14.Spectrum['ylabel'], fontsize=14)

Text(12.444444444444452, 0.5, 'erg cm$^{-2}$ s$^{-1}$ sr$^{-1}$Å$^{-1}$')

this calculates the spectrum of fe_14 over the specified wavelength range and filter it with the default filter which is a gaussian (filters.gaussianR) with a ‘resolving power’ of 1000 which gives a gaussian width of wvl/1000.  Other filters available in chianti.filters include a boxcar filter and a gaussian filter where the gaussian width can be specified directly

the units of the vertical axis is erg cm$^{-2}$ s$^{-1}$ sr$^{-1}$ $\mathring A$$^{-1}$ since the value of the emission measure **em** has been specified

In [75]:
if hasattr(fe14,'Em'):
    print(' Emission Measure = %12.2e'%(fe14.Em[0]))
else:
    print(' the value for the emission measure is unspecified')

 Emission Measure =     1.00e+27


In [76]:
fe14.spectrum(wvl,filter=(chfilters.gaussian,.4))

calculates the spectrum of fe_14 for a gaussian filter with a width of 0.4 Angstroms.  The current value of the spectrum is kept in fe14.Spectrum with the following keys:

In [77]:
for akey in sorted(fe14.Spectrum.keys()):
    print(' %10s'%(akey))

   allLines
         em
     filter
 filterWidth
 integrated
  intensity
 wavelength
     xlabel
     ylabel


In [78]:
if hasattr(fe14,'Em'):
    print(' Emission Measure = %12.2e'%(fe14.Em[0]))
else:
    print(' the value for the emission measure is unspecified')

 Emission Measure =     1.00e+27


### Here, the previously value of the emission measure has been used

In [80]:
plt.figure()
plt.plot(wvl,fe14.Spectrum['intensity'])
plt.xlabel(fe14.Spectrum['xlabel'], fontsize=14)
plt.ylabel(fe14.Spectrum['ylabel'], fontsize=14)

Text(0, 0.5, 'erg cm$^{-2}$ s$^{-1}$ sr$^{-1}$Å$^{-1}$')

In [81]:
xy = plt.axis()
xy

(np.float64(195.0),
 np.float64(305.0),
 np.float64(-11.018591424045812),
 np.float64(231.39042332721723))

In [83]:
plt.axis([wvl[0], wvl[-1], 0., 250.])
plt.tight_layout()

### As of **ChiantiPy 0.14.0**, the **ion** class inherits the spectrumPlot method.

In [88]:
mpl.rcParams['font.size'] = 12

In [89]:
wvlRange = [wvl[0], wvl[-1]]
fe14.spectrumPlot(wvlRange=wvlRange)

## also in 0.14.0 is the saveData method

In [90]:
saveName = 'fe14_save.pkl'

In [91]:
fe14.saveData(saveName, verbose=True)

 saving attribute:  Filename
 saving attribute:  Date
 saving attribute:  ClassName
 saving attribute:  IonStr
 saving attribute:  Z
 saving attribute:  Ion
 saving attribute:  Iso
 saving attribute:  Dielectronic
 saving attribute:  HigherName
 saving attribute:  Spectroscopic
 saving attribute:  FileName
 saving attribute:  Defaults
 saving attribute:  Labels
 saving attribute:  Abundance
 saving attribute:  AbundAll
 saving attribute:  IoneqName
 saving attribute:  RadTemperature
 saving attribute:  RStar
 saving attribute:  Ip
 saving attribute:  FIP
 saving attribute:  IoneqAll
 saving attribute:  Temperature
 saving attribute:  Ntemp
 saving attribute:  ProtonDensityRatio
 saving attribute:  EDensity
 saving attribute:  Ndens
 saving attribute:  NTempDens
 saving attribute:  PDensity
 saving attribute:  Em
 saving attribute:  IoneqOne
 saving attribute:  Elvlc
 saving attribute:  Wgfa
 saving attribute:  Nlvls
 saving attribute:  Nwgfa
 saving attribute:  Scups
 saving attribute:

the attributes are saved as a dict and saved as a pickle file

In [92]:
with open(saveName,'rb') as inpt:
    fe14Dict = pickle.load(inpt)

In [93]:
for akey in fe14Dict:
    print(' key = %s'%(akey))

 key = Filename
 key = Date
 key = ClassName
 key = IonStr
 key = Z
 key = Ion
 key = Iso
 key = Dielectronic
 key = HigherName
 key = Spectroscopic
 key = FileName
 key = Defaults
 key = Labels
 key = Abundance
 key = AbundAll
 key = IoneqName
 key = RadTemperature
 key = RStar
 key = Ip
 key = FIP
 key = IoneqAll
 key = Temperature
 key = Ntemp
 key = ProtonDensityRatio
 key = EDensity
 key = Ndens
 key = NTempDens
 key = PDensity
 key = Em
 key = IoneqOne
 key = Elvlc
 key = Wgfa
 key = Nlvls
 key = Nwgfa
 key = Scups
 key = Nscups
 key = Ncilvl
 key = Nreclvl
 key = Nrrlvl
 key = Psplups
 key = Npsplups
 key = DrParams
 key = RrParams
 key = Nauto
 key = GrndLevels
 key = Wavelength
 key = WvlRange
 key = Upsilon
 key = PUpsilon
 key = popmat
 key = Population
 key = Emiss
 key = Intensity
 key = Spectrum
 key = Error


In [94]:
for akey in fe14Dict['Spectrum']:
    print(' key = %s'%(akey))

 key = intensity
 key = integrated
 key = wavelength
 key = filter
 key = filterWidth
 key = allLines
 key = em
 key = xlabel
 key = ylabel


In [95]:
plt.figure()
plt.plot(fe14Dict['Spectrum']['wavelength'], fe14Dict['Spectrum']['integrated'])

it is possible to work directly with the saved data

## Using emission measures (EM)

Beginning with CHIANTI version 10, a new directory, em, as been added to contain emission measure files.

## the line-of-sight emission measure is give by

$$
  \large{\int \, n_e \, n_H \, dl \, ({cm}^{-5})}
$$

## and the volumetric emission measure is given by

$$
  \large{\int \, n_e \, n_H \, dV \,( cm^{-3})}
$$

### where the integration is over the source region

In [96]:
emDir = os.path.join(os.environ['XUVTOP'], 'em')

In [97]:
emList = os.listdir(emDir)

In [98]:
for idx, emFile in enumerate(emList):
    print('%i  %s'%(idx, emFile))

0  quiet_sun_1993_serts_4T.em
1  active_region_1993_serts_4T.em
2  sunspot_19990316.em
3  sunspot_19990317.em


In [99]:
arDict = chio.emRead(emList[1])

In [100]:
arDict.keys()

dict_keys(['temperature', 'density', 'em', 'ref', 'filename'])

In [101]:
arTemp = arDict['temperature']
arDens = arDict['density']
arEm = arDict['em']

In [102]:
for idx, atemp in enumerate(arTemp):
    print('%i %10.2e %10.2e %10.2e'%(idx, atemp, arDens[idx], arEm[idx]))

0   6.17e+05   2.00e+09   4.97e+26
1   1.12e+06   2.00e+09   2.09e+27
2   1.86e+06   2.00e+09   7.89e+27
3   3.16e+06   2.00e+09   1.46e+28


In [103]:
fe14  = ch.ion('fe_14', arTemp, arDens, em=arEm)

In [104]:
wvl = np.linspace(200., 300., 10001)

In [105]:
fe14.spectrum(wvl, filter=(chfilters.gaussian, .03))

In [108]:
fe14.spectrumPlot(wvlRange=[264., 275.], integrated=True, top=5)

In [110]:
fe14.spectrumPlot(wvlRange=[210., 220.], index=2, top=5, doLabel=False)

### with version 0.14.0, there is a new class, redux

### with this class, the saved data can be restored and all of the apprpriated inherited methods are available

In [111]:
rdx = ch.redux(saveName, verbose=True)

 restored attribute Filename
 restored attribute Date
 restored attribute ClassName
 restored attribute IonStr
 restored attribute Z
 restored attribute Ion
 restored attribute Iso
 restored attribute Dielectronic
 restored attribute HigherName
 restored attribute Spectroscopic
 restored attribute FileName
 restored attribute Defaults
 restored attribute Labels
 restored attribute Abundance
 restored attribute AbundAll
 restored attribute IoneqName
 restored attribute RadTemperature
 restored attribute RStar
 restored attribute Ip
 restored attribute FIP
 restored attribute IoneqAll
 restored attribute Temperature
 restored attribute Ntemp
 restored attribute ProtonDensityRatio
 restored attribute EDensity
 restored attribute Ndens
 restored attribute NTempDens
 restored attribute PDensity
 restored attribute Em
 restored attribute IoneqOne
 restored attribute Elvlc
 restored attribute Wgfa
 restored attribute Nlvls
 restored attribute Nwgfa
 restored attribute Scups
 restored attribut

### the following gives the previous plot

In [113]:
rdx.spectrumPlot(wvlRange=wvlRange)

In ChiantiPy 0.6, the **label** keyword has been added to the ion.spectrum method, and also to the other various spectral classes.  This allows several spectral calculations for different filters to be saved and compared

In [114]:
temp = 10.**(5.8 + 0.1*np.arange(11.))
dens = 1.e+9
emeas = np.ones(11, np.float64)*1.e+27

In [115]:
fe14 = ch.ion('fe_14', temp, dens, em=emeas)

In [116]:
wvl = 200. + 0.125*np.arange(801)
fe14.spectrum(wvl,filter=(chfilters.gaussian,.4),label='.4')
fe14.spectrum(wvl,filter=(chfilters.gaussian,1.),label='1.')

In [117]:
for akey in sorted(fe14.Spectrum.keys()):
    print(' %10s'%(akey))

         .4
         1.


In [118]:
for akey in sorted(fe14.Spectrum['.4'].keys()):
    print(' %10s'%(akey))

   allLines
         em
     filter
 filterWidth
 integrated
  intensity
 wavelength
     xlabel
     ylabel


In [119]:
plt.figure()
plt.plot(wvl,fe14.Spectrum['.4']['intensity'][5],label='0.4')
plt.plot(wvl,fe14.Spectrum['1.']['intensity'][5],'-r', label = '1.0')
plt.axis([wvl[0], wvl[-1], 0., 250.])
plt.xlabel(fe14.Spectrum['.4']['xlabel'], fontsize=14)
plt.ylabel(fe14.Spectrum['.4']['ylabel'], fontsize=14)
plt.legend(loc='upper right', fontsize=14)
plt.tight_layout()

Free-free and free-bound continuum
==================================

The module continuum provides the ability to calculate the free-free, free-bound continuum spectrum for a large number of individual ions.  The two-photon continuum is produced only by the hydrogen-like and helium-like ions

In [120]:
myIon = 'fe_25'

In [121]:
temperature = [2.e+7, 3.e+7, 6.e+7]
density = 1.e+9
em = [1.e+27, 1.e+27, 1.e+27]
wvl = 0.5 + 0.002*np.arange(4501)

In [122]:
c = ch.continuum(myIon, temperature = temperature, em=em)
c.freeFree(wvl)
c.freeBound(wvl)
fe25=ch.ion(myIon, temperature, density, em=em)
fe25.twoPhoton(wvl)
total = c.FreeFree['intensity'] + c.FreeBound['intensity'] + fe25.TwoPhoton['intensity']

In [123]:
itemp = 1
plt.figure()
plt.plot(wvl, c.FreeFree['intensity'][itemp],label='ff')
plt.plot(wvl, c.FreeBound['intensity'][itemp],label='fb')
plt.plot(wvl,fe25.TwoPhoton['intensity'][itemp],label='2 photon')
plt.plot(wvl, total[itemp], 'k', label='total')
plt.xlabel(c.FreeFree['xlabel'], fontsize=14)
plt.ylabel(c.FreeFree['ylabel'], fontsize=14)
plt.legend(loc='upper right', fontsize=14)
plt.title(' %s  T = %10.2e'%(fe25.IonStr, temperature[itemp]), fontsize=14)
plt.ylim(bottom=0.)
plt.xlim([0., wvl[-1]])
plt.tight_layout()

In [124]:
myIon = 'o_8'

In [125]:
temperature = [3.e+6, 6.e+6]
density = 1.e+9
em = [2.e+27,1.e+27]
wvl = 2. + 0.2*np.arange(701)

In [126]:
c = ch.continuum('o_8', temperature = temperature, em=em)
c.freeFree(wvl)
c.freeBound(wvl)
o8 = ch.ion(myIon, temperature, density, em=em)
o8.twoPhoton(wvl)
total = c.FreeFree['intensity'] + c.FreeBound['intensity'] + o8.TwoPhoton['intensity']

In [127]:
itemp = 1
plt.figure()
plt.plot(wvl, c.FreeFree['intensity'][itemp], label='ff')
plt.plot(wvl, c.FreeBound['intensity'][itemp], label='fb')
plt.plot(wvl, o8.TwoPhoton['intensity'][itemp], label='2 photon')
plt.plot(wvl, total[itemp], 'k', label='total')
plt.ylim(bottom=0.)
#plt.xlim(left = 0., right= 100.)
plt.xlabel(c.FreeFree['xlabel'], fontsize=14)
plt.ylabel(c.FreeFree['ylabel'], fontsize=14)
plt.title(' %s  T = %10.2e'%(o8.IonStr, temperature[itemp]), fontsize=14)
plt.legend(loc='upper right', fontsize=14)
plt.tight_layout()

In [128]:
itemp = 0
plt.figure()
plt.semilogy(wvl, c.FreeFree['intensity'][itemp],label='ff')
plt.semilogy(wvl, c.FreeBound['intensity'][itemp],label='fb')
plt.semilogy(wvl,o8.TwoPhoton['intensity'][itemp],label='2 photon')
plt.semilogy(wvl, total[itemp], 'k', label='total')
plt.ylim(bottom=1.e-4)
#plt.xlim(left = 0., right= 100.)
plt.xlabel(c.FreeFree['xlabel'], fontsize=14)
plt.ylabel(c.FreeFree['ylabel'], fontsize=14)
plt.title(' %s  T = %10.2e'%(o8.IonStr, temperature[itemp]), fontsize=14)
plt.legend(loc='upper right', fontsize=14)
plt.tight_layout()

In the continuum calculations, Fe XXV in this case, is the target ion for the free-free calculation. For the free-bound calculation, specified ion is also the target ion. In this case, the radiative recombination spectrum of Fe XXV recombining to form Fe XXIV is returned.  **Note** this will bring up an error message.  Things are probably OK for the present.

In [129]:
for akey in  ChiantiPy.tools.data.Defaults.keys():
   print(' %10s - %s'%(akey,ChiantiPy.tools.data.Defaults[akey]))

  abundfile - sun_photospheric_2021_asplund
  ioneqfile - chianti
 wavelength - angstrom
       flux - energy
        gui - True
     rcfile - /home/ken/.config/chiantirc


The multi-ion class bunch
=========================

The multi-ion class **bunch** [new in v0.6] inherits a number of the same methods inherited by the ion class, for example *intensityList*, *intensityRatio*, and *intensityRatioSave*. As a short demonstration of its usefulness, Widing and Feldman (1989, ApJ, 344, 1046) used line ratios of Mg VI and Ne VI as diagnostics of elemental abundance variations in the solar atmosphere. For that to be accurate, it is necessary that the lines of the two ions have the same temperature response.

In [130]:
temp = 10.**(5.0+0.1*np.arange(11))
bnch=ch.bunch(temp, 1.e+9, wvlRange=[300.,500.], ionList=['ne_6','mg_6'], abundance='unity', em=1.e+27)

 elapsed seconds =        0.000


In [131]:
bnch.intensityRatio(wvlRange=[395.,405.],top=6)

 ndens =     1 ntemp =    11
   403.307 2s2.2p3 4S1.5 - 2s.2p4 4P2.5
   403.260 2s2 2p 2P1.5 - 2s 2p2 2P0.5
   401.941 2s2 2p 2P1.5 - 2s 2p2 2P1.5
   401.146 2s2 2p 2P0.5 - 2s 2p2 2P0.5
   400.662 2s2.2p3 4S1.5 - 2s.2p4 4P1.5
   399.841 2s2 2p 2P0.5 - 2s 2p2 2P1.5


### there seems to be a significant temperature dependence to the ratio, even though both are formed near 4.e+5 K.

## the intensityPlot method can also be used with bunch

In [134]:
bnch.intensityPlot(wvlRange=[398., 404.], index=5, top=7, lw=2)

using index =     5 specifying temperature =    3.16e+05


### with version 0.13.0 it is possible to save multi-ion calculations as a pickle file

In [135]:
dataName = 'mybunch.pkl'
bnch.saveData(dataName, verbose=True)

 saving attribute:  Filename
 saving attribute:  Date
 saving attribute:  ClassName
 saving attribute:  Temperature
 saving attribute:  Ntemp
 saving attribute:  EDensity
 saving attribute:  Ndens
 saving attribute:  NTempDens
 saving attribute:  Em
 saving attribute:  Defaults
 saving attribute:  Labels
 saving attribute:  AbundanceName
 saving attribute:  AbundAll
 saving attribute:  Abundance
 saving attribute:  IonsCalculated
 saving attribute:  Finished
 saving attribute:  WvlRange
 saving attribute:  Todo
 saving attribute:  Intensity
 saving attribute:  IntensityRatio
 saving attribute:  Message
 saving attribute:  Error


the saveData method creates a dict of all of the attributes of the bnch instance. The pickle file can be loaded an it is possible to work directly with the data.

In [136]:
with open(dataName, 'rb') as inpt:
    mybnch = pickle.load(inpt)

In [137]:
mybnch.keys()

dict_keys(['Filename', 'Date', 'ClassName', 'Temperature', 'Ntemp', 'EDensity', 'Ndens', 'NTempDens', 'Em', 'Defaults', 'Labels', 'AbundanceName', 'AbundAll', 'Abundance', 'IonsCalculated', 'Finished', 'WvlRange', 'Todo', 'Intensity', 'IntensityRatio', 'Message', 'Error'])

In [138]:
mybnch['Intensity']['intensity'].shape

(11, 4823)

In [139]:
rebnch = ch.redux(dataName, verbose=False)

In [141]:
rebnch.intensityPlot(index=5, wvlRange=[398., 404.])

using index =     5 specifying temperature =    3.16e+05


with verbose set to True, a list of attributes that are saved is printed

produces the previous figure

### A new keyword argument **keepIons** has been added in v0.6 to the bunch and the 3 spectrum classes.

In [142]:
temp = 10.**(5.0+0.2*np.arange(6))
dens = 1.e+9

In [143]:
dwvl = 0.01
nwvl = (406. - 394.)/dwvl
wvl = 394. + dwvl*np.arange(nwvl + 1)
wvlRange = [wvl.min(), wvl.max()]

In [144]:
bnch2=ch.bunch(temp, dens, wvlRange=wvlRange, elementList=['ne','mg'], keepIons=True, em=1.e+27)

 elapsed seconds =       11.000


it is also possible to create a spectrum with specified line width.

In [145]:
bnch2.convolve(wvl,filter=(chfilters.gaussian, 5.*dwvl))

 elapsed seconds =    8.000e+00


In [146]:
for one in sorted(bnch2.IonInstances.keys()):
    print('%s'%(one))

mg_10
mg_11
mg_12
mg_2
mg_3
mg_4
mg_5
mg_6
mg_7
mg_8
mg_9
ne_1
ne_10
ne_2
ne_3
ne_4
ne_5
ne_6
ne_7
ne_8
ne_9


In [148]:
bnch2.spectrumPlot(integrated=True, doLabel=False)
plt.plot(wvl,bnch2.IonInstances['mg_6'].Spectrum['integrated'], 'r', label='mg_6')
plt.xlim(left=398., right=404.)
plt.legend(loc='upper left', fontsize=14)

In [149]:
bnch2.spectrumPlot(top=7)

Spectra of multiple ions and continuum
======================================

The spectrum for a selection of all of the ions in the CHIANTI database can also be calculated. There are 3 spectral classes.

1.  spectrum - the single processor implementation that can be used anywhere
2.  mspectrum - uses the Python multiprocessing class and **cannot** be used in a IPython qtconsole or notebook
3.  ipymspectrum [new in v0.6] - uses the IPython parallel class and can be used in a IPython qtconsole or notebook


spectrum, mspectrum and ipymspectrum can all be instantiated with the same arguments and keyword arguments.  Most of the examples below use the ipymspectrum class for speed.

As of version 0.13.0, it is now possible to save the calculations with the saveData methods, demonstrated with the bunch class above

The single processor spectrum class,  this may take a while
----------------------------------------------------

In [159]:
temp = [1.e+6, 2.e+6]
dens = 1.e+9
wvl = 200. + 0.05*np.arange(2001)
emeasure = [1.e+27 ,1.e+27]

In [160]:
s = ch.spectrum(temp, dens, wvl, filter = (chfilters.gaussian,.2), em = emeasure, doContinuum = False, minAbund=1.e-5)

 elapsed seconds =       78.000


In [161]:
plt.figure()
plt.subplot(311)
plt.plot(wvl, s.Spectrum['integrated'])
plt.ylim(bottom=0.)
plt.title('integrated')
plt.subplot(312)
plt.plot(wvl, s.Spectrum['intensity'][0])
plt.ylim(bottom=0.)
plt.subplot(313)
plt.plot(wvl, s.Spectrum['intensity'][1])
plt.ylim(bottom=0.)

(0.0, 1037.584829321073)

### using the matplotlib object scheme

In [162]:
fig, ax = plt.subplots(ncols = 1, nrows = 3, tight_layout = True)
ax[0].plot(wvl, s.Spectrum['integrated'], lw = 2)
ax[0].set_ylim(bottom = 0.)
ax[0].set_title('Integrated')
ax[1].plot(wvl, s.Spectrum['intensity'][0], lw = 2)
ax[1].set_ylim(bottom = 0.)
ax[2].plot(wvl, s.Spectrum['intensity'][1], lw = 2)
ax[2].set_ylim(bottom = 0.)

(0.0, 1037.584829321073)

### some adjustments are probably needed to get a proper figure

### save the calculations

In [163]:
saveName = 'spectrum.pkl'

In [164]:
s.saveData(saveName, verbose=True)

 saving attribute:  Filename
 saving attribute:  Date
 saving attribute:  ClassName
 saving attribute:  Defaults
 saving attribute:  Wavelength
 saving attribute:  WvlRange
 saving attribute:  Temperature
 saving attribute:  Ntemp
 saving attribute:  EDensity
 saving attribute:  Ndens
 saving attribute:  NTempDens
 saving attribute:  Em
 saving attribute:  Labels
 saving attribute:  AbundanceName
 saving attribute:  AbundAll
 saving attribute:  Abundance
 saving attribute:  MinAbund
 saving attribute:  IonsCalculated
 saving attribute:  Finished
 saving attribute:  Todo
 saving attribute:  Intensity
 saving attribute:  FreeFree
 saving attribute:  FreeBound
 saving attribute:  LineSpectrum
 saving attribute:  TwoPhoton
 saving attribute:  Continuum
 saving attribute:  Total
 saving attribute:  Spectrum


In [166]:
mpl.rcParams['font.size'] = 12.0

In [167]:
s.spectrumPlot(integrated=True)

In [168]:
mpl.rcParams['font.size'] = 16.0

## one can return to the saved data and reload it with the redux class

In [169]:
rdx = ch.redux(saveName)

In [171]:
mpl.rcParams['font.size'] = 12.0

In [172]:
rdx.spectrumPlot(index=1)

In [173]:
mpl.rcParams['font.size'] = 16.0

The multiple processor mspectrum class
-----------------------------------------------------------

### the class uses the Python multiprocessing module

In [150]:
temp = [1.e+7, 2.e+7, 3.e+7]
dens = 1.e+9
wvl = np.linspace(1.5, 4., 10001)
emeasure = 1.e+27

In [151]:
dwvl = wvl[1] - wvl[0]
' dwvl:  %8.4f'%(dwvl)

' dwvl:    0.0003'

In [152]:
core = 8

In [153]:
sm = ch.mspectrum(temp, dens, wvl, filter = (chfilters.gaussian, 5.*dwvl), em = emeasure, doContinuum=False,
                  minAbund=1.e-5, proc=core)

 elapsed seconds =        7.000


In [154]:
sm.spectrumPlot(wvlRange=[1.84, 1.88], integrated=True)

### save the calculations

In [181]:
saveName = 'mspectrum.pkl'

In [182]:
sm.saveData(saveName, verbose=True)

 saving attribute:  Filename
 saving attribute:  Date
 saving attribute:  ClassName
 saving attribute:  Defaults
 saving attribute:  Temperature
 saving attribute:  Ntemp
 saving attribute:  EDensity
 saving attribute:  Ndens
 saving attribute:  NTempDens
 saving attribute:  Em
 saving attribute:  Labels
 saving attribute:  AllLines
 saving attribute:  MinAbund
 saving attribute:  AbundanceName
 saving attribute:  Abundance
 saving attribute:  AbundAll
 saving attribute:  Wavelength
 saving attribute:  WvlRange
 saving attribute:  IonsCalculated
 saving attribute:  Finished
 saving attribute:  Todo
 saving attribute:  Intensity
 saving attribute:  FreeFree
 saving attribute:  FreeBound
 saving attribute:  LineSpectrum
 saving attribute:  TwoPhoton
 saving attribute:  Continuum
 saving attribute:  Spectrum
 saving attribute:  Error


## another example

## Using differential emission measures (DEM)

Beginning with CHIANTI version 14, the io.demRead function has been added
to read dem file in the existing XUVTOP/dem directory

In [195]:
demDir = os.path.join(os.environ['XUVTOP'], 'dem')

In [196]:
demList = os.listdir(demDir)

In [197]:
for idx, demFile in enumerate(demList):
    print('%i  %s'%(idx, demFile))

0  quiet_sun_eis.dem
1  flare.dem
2  quiet_sun.dem
3  prominence.dem
4  AU_Mic.dem
5  version_3
6  flare_ext.dem
7  active_region.dem
8  coronal_hole.dem


select the desired file by index

In [198]:
flDict = chio.demRead(demList[6])

In [199]:
flDict.keys()

dict_keys(['temperature', 'density', 'dem', 'em', 'dt', 'ref', 'filename'])

since we will be looking at X-ray wavelengths, select only the highest temperatures

In [200]:
flTemp = flDict['temperature'][20:]
flDens = flDict['density'][20:]
flEm = flDict['em'][20:]

In [201]:
for idx, atemp in enumerate(flTemp):
    print('%3i %10.2e %10.2e %10.2e'%(idx, atemp, flDens[idx], flEm[idx]))

  0   3.16e+06   1.00e+10   1.25e+29
  1   3.98e+06   1.00e+10   2.83e+29
  2   5.01e+06   1.00e+10   7.59e+29
  3   6.31e+06   1.00e+10   2.00e+30
  4   7.94e+06   1.00e+10   4.59e+30
  5   1.00e+07   1.00e+10   8.75e+30
  6   1.26e+07   1.00e+10   1.28e+31
  7   1.58e+07   1.00e+10   1.13e+31
  8   2.51e+07   1.00e+10   9.17e+30
  9   5.01e+07   1.00e+10   7.28e+30
 10   1.00e+08   1.00e+10   7.28e+30


In [202]:
flDict['ref']

['%filename: flare.dem\n',
 '%dem: Dere, K.P., Cook, J.W., 1979, ApJ, 229, 772\n',
 '%comment: composite of August 9 1553 and 1554 UT data of an M2 X-ray class flare\n',
 '%comment: modifies at high temperature (7.3 to 8.0) by G.Del Zanna to calculate \n',
 '          the emissivities of the hottest ions.\n',
 "% produced as part of the Arcetri/Cambridge/NRL 'CHIANTI' atomic data base collaboration\n",
 '%\n',
 '%   K.P.Dere and G. Del Zanna - Aug 2002\n',
 ' -1\n']

In [203]:
wvl = 1. + 0.002*np.arange(4501)

In [206]:
core = 8

In [207]:
s3 = ch.mspectrum(flTemp, flDens, wvl, filter = (chfilters.gaussian, .015), doContinuum=1, em=flEm, minAbund=1.e-5,
                  verbose=0, proc=core)

 elapsed seconds =       14.000


### save the calculations

In [208]:
saveName = 'mspectrum3_dem.pkl'

In [209]:
s3.saveData(saveName, verbose=True)

 saving attribute:  Filename
 saving attribute:  Date
 saving attribute:  ClassName
 saving attribute:  Defaults
 saving attribute:  Temperature
 saving attribute:  Ntemp
 saving attribute:  EDensity
 saving attribute:  Ndens
 saving attribute:  NTempDens
 saving attribute:  Em
 saving attribute:  Labels
 saving attribute:  AllLines
 saving attribute:  MinAbund
 saving attribute:  AbundanceName
 saving attribute:  Abundance
 saving attribute:  AbundAll
 saving attribute:  Wavelength
 saving attribute:  WvlRange
 saving attribute:  IonsCalculated
 saving attribute:  Finished
 saving attribute:  Todo
 saving attribute:  Intensity
 saving attribute:  FreeFree
 saving attribute:  FreeBound
 saving attribute:  LineSpectrum
 saving attribute:  TwoPhoton
 saving attribute:  Continuum
 saving attribute:  Spectrum


In [210]:
plt.figure()
plt.plot(wvl, s3.Spectrum['intensity'].sum(axis=0))
plt.xlabel(s3.Spectrum['xlabel'], fontsize=14)
plt.ylabel(s3.Spectrum['ylabel'], fontsize=14)
plt.ylim(bottom = 0.)
plt.xlim([0., wvl[-1]])
plt.tight_layout()

In [211]:
s3.spectrumPlot(top=6, integrated=True)

with doContinuum=1, the continuum can be plotted separately - the default value for doContinuum = True/1

In [212]:
plt.figure()
plt.plot(wvl, s3.FreeFree['intensity'].sum(axis=0), label='FF')
plt.plot(wvl, s3.FreeBound['intensity'].sum(axis=0), label='FB')
plt.plot(wvl, s3.TwoPhoton['intensity'].sum(axis=0), label='2 Photon')
plt.plot(wvl, s3.Continuum['intensity'].sum(axis=0), 'k', label='Total')
plt.xlabel(s3.Spectrum['xlabel'], fontsize=14)
plt.ylabel(s3.Spectrum['ylabel'], fontsize=14)
plt.ylim(bottom = 0.)
plt.xlim([0., wvl[-1]])
plt.legend(loc='upper right', fontsize=14)
plt.tight_layout()

In [213]:
s3.spectrumPlot(wvlRange=[4., 9.], top=6, integrated=True)

### with the redux class, the save calculations can be restored

In [216]:
s3r = ch.redux(saveName, verbose=True)

 restored attribute Filename
 restored attribute Date
 restored attribute ClassName
 restored attribute Defaults
 restored attribute Temperature
 restored attribute Ntemp
 restored attribute EDensity
 restored attribute Ndens
 restored attribute NTempDens
 restored attribute Em
 restored attribute Labels
 restored attribute AllLines
 restored attribute MinAbund
 restored attribute AbundanceName
 restored attribute Abundance
 restored attribute AbundAll
 restored attribute Wavelength
 restored attribute WvlRange
 restored attribute IonsCalculated
 restored attribute Finished
 restored attribute Todo
 restored attribute Intensity
 restored attribute FreeFree
 restored attribute FreeBound
 restored attribute LineSpectrum
 restored attribute TwoPhoton
 restored attribute Continuum
 restored attribute Spectrum


### the redux class inherits the intensityPlot and spectrumPlot methods as well as a few others

In [217]:
s3r.spectrumPlot(wvlRange=[6., 7.], integrated=True, top=5)

The multiple processor ipymspectrum class
-----------------------------------------

next, we will use the ipymspectrum class.  First, it is necessary to start up the cluster.  In some shell

> ipcluster start --n=4

this will start 4 engines if you have 4 cores.  It will only start an many engines as there are cores available

In [220]:
temp = [1.e+6, 2.e+6]
dens = 1.e+9
wvl = 200. + 0.05*np.arange(2001)
emeasure = [1.e+27 ,1.e+27]

the following code may raise some RuntimeWarnings like

IPython could not determine IPs ..., or 

Controller appears to be listening on localhost, but not on this machine ...

but the code does actually run

In [221]:
s = ch.ipymspectrum(temp, dens, wvl, filter = (chfilters.gaussian,.2), em = emeasure, doContinuum=False,
                    minAbund=1.e-5, verbose=True)

in some terminal 
 >ipcluster start --n=4 , or some other integer
 doing ion c_3 for the following processes line
 doing ion c_4 for the following processes line
 doing ion c_5 for the following processes line
 doing ion c_6 for the following processes line
 doing ion fe_10 for the following processes line
 doing ion fe_11 for the following processes line
 doing ion fe_12 for the following processes line
 doing ion fe_13 for the following processes line
 doing ion fe_14 for the following processes line
 doing ion fe_15 for the following processes line
 doing ion fe_16 for the following processes line
 doing ion fe_17 for the following processes line
 doing ion fe_18 for the following processes line
 doing ion fe_19 for the following processes line
 doing ion fe_20 for the following processes line
 doing ion fe_21 for the following processes line
 doing ion fe_22 for the following processes line
 doing ion fe_6 for the following processes line
 doing ion fe_7 for the following processes

In [222]:
plt.figure()
plt.plot(wvl, s.Spectrum['integrated'])
plt.ylim(bottom=0.)
plt.xlim([wvl[0], wvl[-1]])
plt.title('Integrated')
plt.xlabel(s.Spectrum['xlabel'], fontsize=14)
plt.ylabel(s.Spectrum['ylabel'], fontsize=14)
plt.tight_layout()

Using a 4 core processor running at about 3.5 GHz:
* For a minAbund of 1.e-4, the spectra of 25 ions were calculated in 68 s. On a single processor it took 118 s.
* For a minAbund of 1.e-5, then the spectra of 100 ions would be calculated in 72 s. The previous example with the single processor spectrum class took about  129 s on the same computer.
* For a minAbund of 1.e-6, then the spectra of 170 ions would be calculated in 90 s.  On a single processor it took 183 s.

Using only 3 cores:
* For a minAbund of 1.e-5, then the spectra of 100 ions would be calculated in 73 s.

Using only 2 cores:
* For a minAbund of 1.e-5, then the spectra of 100 ions would be calculated in 88 s.

Using only 1 core, with ch.spectrum:
* For a minAbund of 1.e-5, then the spectra of 100 ions would be calculated in 133 s.

[New in version 0.6] One can also use a different abundance file than the default by specifying the abundanceName keyword. For example, abundanceName = ‘cosmic_1973_allen.abund’. If the specified file is not found in XUVTOP/abundance, then a widget will pop up and one can select the abundance file from a list.

It is also possible to specify a selection of ions by means of the ionList keyword, for example, ionList=[‘fe_11’,’fe_12’,’fe_13’] or with the elementList keyword, for example, elementList=[‘mg’,’si’]


In [223]:
s2 = ch.ipymspectrum(temp, dens, wvl, filter = (chfilters.gaussian,.2), em = emeasure,
                     doContinuum=0, keepIons=True, elementList=['si'])

in some terminal 
 >ipcluster start --n=4 , or some other integer
 elapsed seconds =    7.000e+00


Because **keepIons** has been set, the ion instances of all of the ions are maintained in the s2.IonInstances dictionary. It has been possible to compare the spectrum of all of the ions with the spectrum of a single ion.

In [224]:
fig, [ax1, ax2] = plt.subplots(2,1)
ax1.plot(wvl,s2.Spectrum['intensity'][0])
ax1.set_ylim(bottom=0.)
ax1.set_xlim([wvl[0], wvl[-1]])
ax1.set_ylabel(r'erg cm$^{-2}$ s$^{-1}$ sr$^{-1} \AA^{-1}$', fontsize=14)
ax2.plot(wvl,s2.IonInstances['si_9'].Spectrum['intensity'][0])
ax2.set_ylim(bottom=0.)
ax1.set_xlim([wvl[0], wvl[-1]])
ax2.set_ylabel(r'erg cm$^{-2}$ s$^{-1}$ sr$^{-1} \AA^{-1}$', fontsize=14)
ax2.set_xlabel(r'Wavelength ($\AA$)', fontsize=14)
ax2.set_title('Si IX', fontsize=14)
fig.tight_layout()

In [225]:
temp = 1.e+7
dens = 1.e+9
wvl = 10. + 0.005*np.arange(2001)

In [226]:
s0 = ch.ipymspectrum(temp, dens, wvl, filter = (chfilters.gaussian,.015), elementList=['fe'], em=1.e+27)

in some terminal 
 >ipcluster start --n=4 , or some other integer
 elapsed seconds =    7.000e+00


In [227]:
s0.spectrumPlot(wvlRange=[12., 18.])

In [228]:
temp = 2.e+7
dens = 1.e+9
wvl = 1. + 0.002*np.arange(4501)
s = ch.ipymspectrum(temp, dens, wvl, filter = (chfilters.gaussian,.015),doContinuum=True,
                    em=1.e+27,minAbund=1.e-5,verbose=0)

in some terminal 
 >ipcluster start --n=4 , or some other integer
 elapsed seconds =    8.000e+00


In [229]:
fig, ax = plt.subplots(tight_layout = True)
ax.plot(wvl, s.Spectrum['intensity'])

In [230]:
temp=5.e+6
dens=1.e+9
dwvl = .005
nwvl = (50. - 1.)/dwvl
wvl=1. + dwvl*np.arange(nwvl+1)

by setting abundanceName to 1, a widget with a list of possible abundance sets will pop up and allow you to select one.  Alternative, you can set abundanceName to the name of one of the abundance file (without the .abund suffix) and it will use that set.

In [231]:
s3 = ch.ipymspectrum(temp, dens, wvl, filter = (chfilters.gaussian,5.*dwvl),doContinuum=1, em=1.e+40,minAbund=1.e-5)
plt.figure()
plt.plot(wvl, s3.Spectrum['intensity'])

in some terminal 
 >ipcluster start --n=4 , or some other integer
 elapsed seconds =    1.700e+01


In [232]:
print(' number of ions calculated %i '%(len(s3.IonsCalculated)))

 number of ions calculated 218 


In [233]:
s3.spectrumPlot()

Since the continuum was calculated, it can be plotted separately

In [234]:
plt.figure()
plt.plot(wvl, s3.FreeFree['intensity'],label='FF')
plt.plot(wvl, s3.FreeBound['intensity'],label = 'FB')
plt.plot(wvl, s3.TwoPhoton['intensity'], label = 'TP')
plt.plot(wvl,s3.FreeBound['intensity']+s3.FreeFree['intensity']+s3.TwoPhoton['intensity'],label='FF+FB+TP')
plt.legend(loc='upper right')

In [235]:
temperature = 2.e+7
density = 1.e+9
em = 1.e+27
wvl = 1.84 + 0.0001*np.arange(601)
s4 = ch.ipymspectrum(temperature, density ,wvl, filter = (chfilters.gaussian,.0003), em=em, minAbund=1.e-5,verbose=0)

in some terminal 
 >ipcluster start --n=4 , or some other integer
 elapsed seconds =    7.000e+00


In [236]:
s4.spectrumPlot(wvlRange=[1.845, 1.88])

# Radiative loss rate

In [237]:
temp = 10.**(4.+0.05*np.arange(81))
dens = 1.e+9
rl = ch.radLoss(temp, dens, elementList=['h', 'he'])

 elapsed seconds =   0.00e+00


In [238]:
plt.figure()
rl.radLossPlot()

### save the data

In [239]:
saveName = 'rl_h_he.pkl'

In [240]:
rl.saveData(saveName)

### with version 0.15.0, the class mradLoss is available for doing a multiprocessor calculation
### of the radiation loss

In [241]:
temp = 10.**(4.+0.05*np.arange(81))
dens = 1.e+9

the following will calculate the radiation loss for elements with an abundance greater the 1.e-5 that of the hydrogen abundance.  In this case the default abundance file is for photospheric abundances

In [242]:
mrl = ch.mradLoss(temp, dens, minAbund = 1.e-5, proc=core)

 elapsed seconds =   6.70e+01


## save the calculations

In [244]:
saveNamePhot = 'rl_phot_1m5.pkl'

In [245]:
mrl.saveData(saveNamePhot)

In [249]:
plt.figure()
mrl.radLossPlot()

### the radiative losses are stored in the dictionary rl.RadLoss

In [250]:
for akey in sorted(mrl.RadLoss.keys()):
    print(' %s '%(akey))

 abundance 
 density 
 minAbund 
 rate 
 temperature 
 xlabel 
 ylabel 


### it is also possible to set the keyword argument **abundance** to use different sets of abundances

In [251]:
abundDir = os.path.join(os.environ['XUVTOP'], 'abundance')

In [252]:
abundList = os.listdir(abundDir)

In [253]:
for idx, aname in enumerate(abundList):
    print('%5i  %s'%(idx, aname))

    0  sun_photospheric_2015_scott.abund
    1  unity.abund
    2  sun_coronal_2021_chianti.abund
    3  sun_photospheric_2009_lodders.abund
    4  sun_photospheric_2007_grevesse.abund
    5  version_3
    6  sun_photospheric_2021_asplund.abund
    7  archive


### to select coronal abundances

In [254]:
cdx = 2

In [255]:
myAbund = abundList[cdx]

In [256]:
mrl2 = ch.mradLoss(temp, dens, abundance=myAbund, minAbund=1.e-4, proc = core, verbose=True)

 doing ion c_1 for the following processes line
 doing ion c_2 for the following processes line_ff_fb
 doing ion c_3 for the following processes line_ff_fb
 doing ion c_4 for the following processes line_ff_fb
 doing ion c_5 for the following processes line_ff_fb
 doing ion c_6 for the following processes line_ff_fb
 doing ion c_7 for the following processes ff_fb
 doing ion h_1 for the following processes line
 doing ion h_2 for the following processes ff_fb
 doing ion he_1 for the following processes line
 doing ion he_2 for the following processes line_ff_fb
 doing ion he_3 for the following processes ff_fb
 doing ion mg_10 for the following processes line_ff_fb
 doing ion mg_11 for the following processes line_ff_fb
 doing ion mg_12 for the following processes line_ff_fb
 doing ion mg_13 for the following processes ff_fb
 doing ion mg_2 for the following processes line_ff_fb
 doing ion mg_3 for the following processes line_ff_fb
 doing ion mg_4 for the following processes line_ff_f

In [257]:
saveNameCoronal = 'rl_coronal_1m4.pkl'

In [258]:
mrl2.saveData(saveNameCoronal)

In [259]:
plt.figure()
mrl2.radLossPlot()

### this took 37s using  8 cores

In [260]:
plt.figure()
plt.loglog(temp, mrl2.RadLoss['rate'], 'k', label='Total')
plt.loglog(temp, mrl2.BoundBoundLoss, label = 'BB')
plt.loglog(temp, mrl2.FreeFreeLoss, label = 'FF')
plt.loglog(temp, mrl2.FreeBoundLoss, label = 'FB')
plt.loglog(temp, mrl2.TwoPhotonLoss, label = '2P')
plt.xlabel(mrl2.RadLoss['xlabel'], fontsize=14)
plt.ylabel(mrl2.RadLoss['ylabel'], fontsize=14)
plt.legend(loc='lower center', fontsize=14)
plt.tight_layout()

### to compare photospheric and coronal radiation losses

In [261]:
rlph = ch.redux(saveNamePhot)

In [262]:
rlco = ch.redux(saveNameCoronal)

In [263]:
plt.figure()
plt.loglog(temp, rlph.RadLoss['rate'], label='Phot')
plt.loglog(temp, rlco.RadLoss['rate'], label='Coronal')
plt.ylim(bottom=1.e-23, top=2.e-21)
plt.xlabel(rlph.RadLoss['xlabel'], fontsize=14)
plt.ylabel(rlph.RadLoss['ylabel'], fontsize=14)
plt.legend(loc='upper right', fontsize=14)
plt.tight_layout()